# Importing important modules and setting up the project

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fare-classification/meta_data.csv
/kaggle/input/my-sample-data/test.csv
/kaggle/input/my-sample-data/train.csv


Reading the data from files

In [2]:
train = pd.read_csv('../input/my-sample-data/train.csv')
test = pd.read_csv('../input/my-sample-data/test.csv')

In [3]:
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [4]:
predictor_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X = train[predictor_cols]
test_X = test[predictor_cols]

#train_y.fillna(train_y.mean(), inplace=True)
train_X.fillna(train_X.mean(), inplace=True)
test_X.fillna(test_X.mean(), inplace=True)
train['num_label'] = 0

for index, row in train.iterrows():
    if(row['label'] == "correct"):
        train.at[index, 'num_label'] = 1
    else:
        train.at[index, 'num_label'] = 0

train_y = train.num_label

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [5]:
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size = 0.2, train_size = 0.8, random_state = 0)

In [6]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    print('Model Performance')
    print('Accuracy:%f'%accuracy)
    print("Mean cross validation score:%f"%score.mean())
    print('Mean Squared Error : %.4f' % mse)
    print('Root MSE : %.4f' % rmse)

Random forest model

In [7]:
my_random_forest_model = RandomForestRegressor()
my_random_forest_model.fit(train_X, train_y)

evaluate(my_random_forest_model, val_X, val_y)

Model Performance
Accuracy:0.945285
Mean cross validation score:0.430273
Mean Squared Error : 0.0427
Root MSE : 0.2066


Decision tree model

In [8]:
my_decision_tree_model = DecisionTreeRegressor()
my_decision_tree_model.fit(train_X, train_y)

evaluate(my_decision_tree_model, val_X, val_y)

Model Performance
Accuracy:0.925204
Mean cross validation score:-0.044218
Mean Squared Error : 0.0748
Root MSE : 0.2735


In [9]:
predicted_rf = my_random_forest_model.predict(test_X)
predicted_dt = my_decision_tree_model.predict(test_X)

output_rf = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_rf})

output_dt = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_dt})
output_rf_path = "submission_rf.csv"
output_dt_path = "submission_dt.csv"

output_rf.to_csv(output_rf_path, index=False)
output_dt.to_csv(output_dt_path, index=False)